In [1]:
import numpy as np
import gurobipy as grb
GRB = grb.GRB

## Add variables

In [2]:
def add_variables(m, size):
    v = {}
    for i in range(size):
        for j in range(size):
            v[(i,j)] = m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name='v'+str(i)+str(j))
    return v

## integrity check

In [3]:
def check_input(size, therms):
    cells = np.zeros((size, size))
    for therm in therms:
        for cell in therm:
            if cells[cell] > 0:
                print('Cell {} used more than once'.format(cell))
            cells[cell] += 1
    for row in range(size):
        for col in range(size):
            if cells[(row, col)] != 1:
                print('Cell ({},{}) has value {}'.format(row, col, cells[(row, col)]))

## Add basic constraints

In [4]:
def add_row_sum_constraints(m, size, v, row_sums):
    for (idx, rsum) in enumerate(row_sums):
        if rsum is None:
            continue
        expr = grb.quicksum([v[idx, c] for c in range(size)])
        m.addConstr(expr, '==', rsum, name='rsum' + str(idx))
def add_col_sum_constraints(m, size, v, col_sums):
    for (idx, csum) in enumerate(col_sums):
        if csum is None:
            continue
        expr = grb.quicksum([v[r, idx] for r in range(size)])
        m.addConstr(expr, '==', csum, name='csum' + str(idx))

## Add thermometer constraints

In [5]:
def add_therm_constraints(m, v, therms):
    for therm in therms:
        for (c1, c2) in zip(therm[:-1], therm[1:]):
            name = ('therm' + str(c1) + str(c2)).replace(' ', '')
            m.addConstr(v[c1], '>=', v[c2], name=name)

## solve function

In [6]:
def show_square(size, v):
    print('\nSolution:\n')
    for i in range(size):
        for j in range(size):
            print(round(v[(i,j)].x), ' ', end='')
        print()

In [7]:
def solve_therm(size, row_sums, col_sums, therms):
    m = grb.Model()
    v = add_variables(m, size)
    add_row_sum_constraints(m, size, v, row_sums)
    add_col_sum_constraints(m, size, v, col_sums)
    add_therm_constraints(m, v, therms)
    m.write('therm.lp')
    m.optimize()
    show_square(size, v)

# Examples:

In [8]:
size = 7
row_sums = [None, None, 2, 4, 3, None, 4]
col_sums = [None, None, 5, 3, 3, None, None]
therms = [
 [(4, 0), (3, 0), (2, 0), (1, 0), (0, 0)],
 [(1, 1), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, 6)],
 [(2, 2), (1, 2), (1, 3), (1, 4), (1, 5), (2, 5), (3, 5), (4, 5)],
 [(2, 3), (2, 4), (3, 4)],
 [(3, 3), (4, 3), (4, 2), (3, 2)],
 [(2, 6), (3, 6), (4, 6), (5, 6), (6, 6)],
 [(4, 4), (5, 4), (5, 3), (5, 2), (5, 1), (4, 1), (3, 1), (2, 1)],
 [(5, 5), (6, 5), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0), (5, 0)]
]
check_input(size, therms)

In [9]:
solve_therm(size, row_sums, col_sums, therms)

Using license file C:\Users\liaml\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 48 rows, 49 columns and 131 nonzeros
Model fingerprint: 0xc16eb7c4
Variable types: 0 continuous, 49 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 5e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

Solution:

0  1  1  0  0  0  0  
0  1  1  0  0  0  0  
0  0  1  0  0  0  1  
1  1  0  1  0  0  1  
1  1  0  0  1  0  0  
0  1  1  1  1  1  0  
0  0  1  1  1  1  0  
